## Agentic AI Tool Calling — How it Works

Agentic AI works even though an LLM only performs next-token prediction because the model never directly executes tools; it only generates structured text that *describes* which tool should be used. The agent program then parses this output and performs the real action externally. In simple terms, the LLM acts as the **planner/brain**, while the program acts as the **executor/hands**.

### Flow

* User sends a query
* LLM predicts structured text (JSON/function call style) indicating which tool to use
* Agent code parses this text
* Actual function/API is executed by the program
* Tool result is returned back to the LLM
* LLM generates the final natural language response

### Key Idea

* LLM → decides **what to do**
* Tools → perform **the action**
* Agent code → connects both

### Common Tool Types

* **API tools** → weather, email, payments, external services
* **Search tools** → web search, database lookup, vector/RAG retrieval
* **Code execution tools** → Python, SQL, calculations
* **File tools** → read/write/summarize PDFs, images, documents
* **Action tools** → schedule meetings, send messages, control devices

### Note

Frameworks like **LangChain** or models from **OpenAI** simply orchestrate this loop internally, but fundamentally it is still just token prediction combined with external code execution.



In [6]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import json

In [7]:
load_dotenv()
openai = OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")
Model = "gpt-4.1-mini"

In [8]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [9]:
def chat(message , history):
    history = [{"role": h["role"] , "content": h["content"]} for h in history]
    messages = [{"role": "system" , "content": system_message}] + history + [{"role": "user" , "content": message}]
    stream = openai.chat.completions.create(model=Model , messages=messages , stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [10]:
gr.ChatInterface(fn=chat , type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


In [11]:
# useful function

tickets = {"london": 100 , "paris": 120 , "new york": 300 , "india" : 400}

def get_ticket_price(destination_city):
    print(f"Tools called for city : {destination_city}")
    price = tickets.get(destination_city.lower(), "Sorry, we don't fly to that city.")
    return f"The ticket price to {destination_city} is {price} rupees." if isinstance(price, int) else price

In [12]:
get_ticket_price("london")

Tools called for city : london


'The ticket price to london is 100 rupees.'

In [13]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a ticket to a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city to which the user wants to fly."
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [14]:
tools = [{"type": "function", "function": price_function}]

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=Model, messages=messages, tools=tools)

    # if response.choices[0].finish_reason=="tool_calls": 
    while response.choices[0].finish_reason=="tool_calls": # this means that the model has called a tool and is waiting for the tool's response before it can continue generating a response to the user.
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=Model, messages=messages)
    
    return response.choices[0].message.content

In [16]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


Tools called for city : India
